# Open double-funded channel without intermediary

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND clients for Ali and Bob

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from p2oc.lnd_rpc import LndRpc, lnmsg

In [ ]:
ali = LndRpc(
    host="ali-lnd:10009",
    tlscertpath="/ali-lnd/tls.cert",
    adminmacaroonpath="/tmp/ali.macaroon",
)

bob = LndRpc(
    host="bob-lnd:10009",
    tlscertpath="/bob-lnd/tls.cert",
    adminmacaroonpath="/tmp/bob.macaroon",
)

## Fund Ali and Bob

In [ ]:
import os
import time

import bitcoin
from p2oc.btc_rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

# Only used to mine initial coins for Ali and Bob
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

brpc.getblockcount()

In [ ]:
# Fund ali and bob
ali_addr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
_ = list(brpc.generatetoaddress(2, ali_addr.address))

bob_addr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
_ = list(brpc.generatetoaddress(2, bob_addr.address))

try:
    brpc.createwallet('miner')
except:
    brpc.loadwallet('miner')

miner_addr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

# Unlock mined coins
_ = list(brpc.generatetoaddress(110, miner_addr))

In [ ]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
# Give bitcoind a chance to catch up
time.sleep(2)

## Perform double-funded channel creation

In [ ]:
import bitcointx.core as bc

import p2oc
from p2oc.psbt import deserialize_psbt

In [ ]:
# Premium Ali (Taker) is willing to pay
premium_amount = int(0.001 * bc.CoreCoinParams.COIN)

# The requested inbound capacity (from Bob the Maker)
fund_amount = int(0.16 * bc.CoreCoinParams.COIN)

In [ ]:
# Ali (Taker) creates offer (to request inbound liquidity in exchange for a fee [the premium])
offer_psbt = p2oc.create_offer(premium_amount, fund_amount, ali)

In [ ]:
# Bob (Maker) accepts offer and sends reply
unsigned_psbt = p2oc.accept_offer(offer_psbt, bob)

In [ ]:
# Ali checks offer reply and opens pending channel
half_signed_psbt = p2oc.open_channel(unsigned_psbt, ali)

In [ ]:
# Bob checks pending channel reply, signs and commits funding tx
p2oc.finalize_offer(half_signed_psbt, bob)

## Confirm channel creation

In [ ]:
_ = list(brpc.generatetoaddress(6, miner_addr))

# Give daemons a chance to catch up
time.sleep(2)

In [ ]:
# Should be no pending channels
bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
bob.lnd.ListChannels(lnmsg.ListChannelsRequest())